In [1]:
import onnxruntime as rt
from skl2onnx import convert_sklearn, update_registered_converter
from skl2onnx.common.shape_calculator import calculate_linear_classifier_output_shapes  # noqa
from onnxmltools.convert.xgboost.operator_converters.XGBoost import convert_xgboost
from skl2onnx.common.data_types import FloatTensorType, DoubleTensorType
import numpy as np
from xgboost import XGBClassifier
from sqlalchemy import create_engine
import sqlalchemy as db
import pandas as pd
import random 

from mclLib.featurenames import *
from mclLib.cleaner import *
from mclLib.server_info import *

In [2]:

engine = create_engine(mysql_server_uri)
conn = engine.connect()

In [3]:
''' get db data '''
br_full_data = pd.read_sql_table('buyreports', conn)

In [4]:

''' db data filter '''
get_filter = ( br_full_data['isAllBuyed'] == 1) & ( br_full_data['isAllSelled'] == 1) 
br = br_full_data[get_filter]

train_test_day_split = '2023-03-01'
train_test_day_filter = (br['dTradeTime'] < train_test_day_split)

''' get features name'''
feature_names_102 =  f_name_102
feature_size = len(feature_names_102)



In [5]:

''' set Train X y data '''
br_train = br[train_test_day_filter]
X_train = br_train[feature_names_102].to_numpy(dtype=np.float64)
y_train_condition = (br_train['fMaxPowerAfterBuyWhile10'] >= 0.04)
y_train = np.where(y_train_condition, 1, 0)


''' set Test X y data '''
br_test = br[~train_test_day_filter]
X_test = br_test[feature_names_102].to_numpy(dtype=np.float64)
y_test_condition = (br_test['fMaxPowerAfterBuyWhile10'] >= 0.04)
y_test = np.where(y_test_condition, 1, 0)


print('X_train : ', X_train.shape)
print('y_train : ', y_train.shape)
print('X_test  : ', X_test.shape)
print('y_test  : ', y_test.shape)

X_train :  (0, 102)
y_train :  (0,)
X_test  :  (14265, 102)
y_test  :  (14265,)


In [6]:
xgbc = XGBClassifier(n_estimators=3)

In [8]:
xgbc.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=3, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [9]:
update_registered_converter(
    XGBClassifier, 'XGBoostXGBClassifier',
    calculate_linear_classifier_output_shapes, convert_xgboost,
    options={'nocl': [True, False], 'zipmap': [True, False, 'columns']})

In [10]:
model_onnx = convert_sklearn(
    xgbc, 'pipeline_lightgbm',options= {'zipmap': False},
    initial_types=[('input', DoubleTensorType([None, 102]))],
    final_types=[('output', DoubleTensorType([None])),
                ('prob', DoubleTensorType([None, 2]))],
    target_opset={'': 17, 'ai.onnx.ml': 2})

# And save.
with open("./onnx/xgboost_test.onnx", "wb") as f:
    f.write(model_onnx.SerializeToString())

In [11]:
cleanAll()